# アドレスはどのように生成されるのか

## アドレスとは
ビットコインをはじめとする仮想通貨を送受するために利用するのがアドレス。 つまり、取引の相手を特定するもの。  
アドレスは多くの人の目に触れるため、可読性を高いもの(公開性)であるが、同時に個人と結びついているプライバシーの高いもの(秘匿性)でもある。  
このような`公開性`と`秘匿性`という一見相反する条件を満たすため、アドレスが生成されるまでにはたくさんの技術が用いられている。

## アドレスが生成されるまでの流れ
アドレスはもとを辿ると秘密鍵に行き着く。  
秘密鍵からアドレスが生成されるまでのプロセスとしては、  秘密鍵が`乱数`として生成された後、それが２回ハッシュ化され、さらにBase58Checkでエンコードされることで生成される。


# 秘密鍵の生成

## 秘密鍵の種は「乱数」
秘密鍵は本質的にはランダムに生成された数である乱数であり、ビットコインのブロックチェーンでは1~2256の間にある整数がその都度ランダムに生成されて秘密鍵となる。  
乱数が本当にランダムに決まっており、規則性を持っていないかというのは非常に重要な点。規則性があるとブロックチェーンにおける資産の所有権そのものである秘密鍵を予測されかねない。

## 秘密鍵を生成してみる
os.urandom(32)の部分で32バイトの乱数を生成。  
binascii.hexlify()では生成したバイナリデータを16進数に変換している。  
その都度、乱数を生成しているため、出力結果は実行ごとに異なる。

In [1]:
import os
import binascii

private_key = os.urandom(32)

print(private_key)
print(binascii.hexlify(private_key))

b"\xdd\x99\xa9\x18r,2;\x19\x94y\xf2\xb93\x96\x11\x05<+\x9e\x1e\xae\x84 |g* i'\xe5\x88"
b'dd99a918722c323b199479f2b9339611053c2b9e1eae84207c672a206927e588'


# 公開鍵の生成

## 公開鍵を生成してみる
公開鍵を生成するのに欠かせない楕円曲線暗号は難易度が高く、脆弱性の懸念もあるので「ecdsa」というサードパーティ製のライブラリを利用して作成する。


In [5]:
import os
import ecdsa
import binascii

# 秘密鍵は乱数で生成
private_key = os.urandom(32)
# 秘密鍵を渡して楕円曲線暗号を利用。from_string()の第一引数は秘密鍵、第二引数は楕円曲線を指定している。
public_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1).verifying_key.to_string()

print(binascii.hexlify(private_key))
print(binascii.hexlify(public_key))

b'd957682699049a73f5ea05829dbe088180b4bce555031db92fc1c05ede51adec'
b'cbfa50d189be683d317299af0928c849f6fdf37a9590eda538ea56a0e9f835f974cf3c7dd3b7f4a4407d010a3a2b755e0a129f5770212039e349197bb665012f'


## 楕円曲線(発展)
公開鍵は秘密鍵とは違い、多くの人に公開される。そのため、公開鍵から秘密鍵が逆算できてしまっては意味がない。  
そこで、公開鍵を生成するために「楕円曲線暗号」と呼ばれる非常に高度な暗号技術を利用する。  
楕円曲線記号はハッシュ関数と同様に一方向性(不可逆製)を持っており、簡単には逆算できないようになっている。  
楕円曲線暗号は楕円曲線と呼ばれる数式を使う。： y² = x³ + ax + b  
aとbの値によって、この曲線は変化しますが、ビットコインのブロックチェーンの場合は、a=0, b=7とした式を利用している。  
この式は上のコードでは secp256k1 と定義されている。  

## 楕円曲線暗号(発展)
楕円曲線暗号に利用される式は  y² = x³ + 7modₚ と表される。  
mod は剰余演算(モジュロ)と呼ばれる記号のことで余りを表す。7mod3 は 1 を表すので、上の式では x³ + 7 を p で割った余りが y² と等しい事になる。  
これは離散対数問題の、ある定数g と素数p がわかっている時、 y = gxmodp の式において、yを求めるのは簡単だが yからxを求めるのは困難だという性質を応用しているもの。

## 秘密鍵から公開鍵を生成(発展)
まず、楕円曲線である y² = x³ + ax + bmodp のパラメータである a,b,p と、基準点となるG(x,y) を設定。  
ビットコインで使っている secp256k1 では既に a, b には値が入っており、ビットコインの場合は基準点Gやpの値も予め決められている。  
公開鍵の生成には、楕円曲線での足し算の性質を利用する。  （楕円曲線上での足し算は、通常の足し算や掛け算とは違い、楕円曲線上の接線を利用するもので、楕円曲線状のある点Gを特定してその点に関する接線を引くと、曲線の性質上、接戦は必ず接点G以外の楕円曲線状の点とぶつかるというもの。そのぶつかった点とx軸に対象に位置する点が2Gとなる。）  
求めた2Gに対してこれを秘密鍵の値の分だけ繰り返すことで得られた値が公開鍵になる。  
この作業を膨大な回数行うことで、最終的な点nGを得ることができ、これが公開鍵として利用される。点nGの計算には膨大な回数が必要となるためコンピューターによる演算が必要。
また、modを利用しているので試行回数が多ければ多いほど、最終的な点nGを特定することが限りなく困難であるとされている。  
特にこの場合は、試行回数んが特定されると、秘密鍵を特定されるのと同じことであるのでこの性質は暗号化技術として非常に重要なことだと言える。

## 公開鍵のフォーマット
公開鍵は楕円曲線暗号を利用して導かれるが、その本質は座標であり、公開鍵は楕円曲線暗号上の点によって定義されているため、x座標とy座標のセットから成り立っている。  
上で作成した公開鍵は秘密鍵と比べて約２倍のサイズがあり、ブロックチェーン全体のデータ容量を考えると圧縮した方が好ましいと考えられていました。  
そこで、公開鍵のフォーマットとして非圧縮公開鍵と圧縮公開鍵の２種類が導入されることとなる。  
##### 非圧縮公開鍵
非圧縮公開鍵は上で導かれた公開鍵に対し、`プレフィックス`として 04 を付加したもの。(プレフィックスとは接頭辞のことでデータの先頭に置かれた特定の意味を持たされた文字列のこと)  
##### 圧縮公開鍵
圧縮公開鍵は公開鍵が座標の情報であることを利用し、関数上の点は x か y がわかれば、もう片方を計算することができるもの。しかし、yが2乗されているため、xの値１つにつき、yの値が２つ求められてしまう。そこで、１つの座標に対してyの座標を１つ特定するためにプレフィックスで区別する。y の値が正の数なら 02 を、負の数なら 03　をつけて区別する。  これにより、非圧縮公開鍵よりも約半分のサイズに圧縮が可能。

### 圧縮公開鍵の生成

In [9]:
import os
import ecdsa
import binascii

private_key = os.urandom(32)
public_key = ecdsa.SigningKey.from_string(private_key, curve=ecdsa.SECP256k1).verifying_key.to_string()

# y座標を取り出す
public_key_y = int.from_bytes(public_key[32:], 'big')

# 圧縮公開鍵を生成
if public_key_y % 2:
    public_key_compressed = b'\x02' + public_key[:32]
else:
    public_key_compressed = b'\x03' + public_key[:32]

print(binascii.hexlify(public_key))
print(binascii.hexlify(public_key_compressed))

b'6ffcdff5c2ebf917547add63fe9f41fdb5872f0a573f29544fc7b3082b2ef31b25193f53e8f1c3bb1cb4a4f8faa4ae70c3b606f51a2788e2593e18f16f6f977f'
b'026ffcdff5c2ebf917547add63fe9f41fdb5872f0a573f29544fc7b3082b2ef31b'
